# Korean OCR with PaddleOCR

이 노트북은 PaddleOCR을 사용하여 한국어 텍스트 인식을 수행합니다.

In [1]:
import sys
import os

# 환경 정보 확인
print(f"🐍 Python 경로: {sys.executable}")
print(f"📁 작업 디렉토리: {os.getcwd()}")

# PATH 확인 (처음 3개만)
path_list = os.environ['PATH'].split(':')
print(f"\n📋 PATH 순서:")
for i, path in enumerate(path_list[:3], 1):
    marker = "✅" if "meow-chat" in path else "📍"
    print(f"  {i}. {marker} {path}")

# 프로젝트 루트 경로 추가
sys.path.append('/home/aidan/work/meow-chat')

try:
    # 기본 패키지 import 테스트
    import cv2
    import numpy as np
    import matplotlib.pyplot as plt
    from PIL import Image
    import pyparsing
    from paddleocr import PaddleOCR
    print("\n✅ 기본 패키지 import 성공!")
    
    # 프로젝트 모듈 import
    from app.services.ocr.paddle_ocr import MyPaddleOCR
    print("✅ MyPaddleOCR import 성공!")
    
except ImportError as e:
    print(f"\n❌ Import 오류: {e}")

print("\n🎯 모든 설정이 완료되었습니다!")

🐍 Python 경로: /home/aidan/miniconda3/envs/meow-chat/bin/python
📁 작업 디렉토리: /home/aidan/work/meow-chat/notebooks/ocr

📋 PATH 순서:
  1. ✅ /home/aidan/miniconda3/envs/meow-chat/bin
  2. 📍 /home/aidan/.vscode-server/bin/385651c938df8a906869babee516bffd0ddb9829/bin/remote-cli
  3. 📍 /home/aidan/.local/bin

✅ 기본 패키지 import 성공!
✅ MyPaddleOCR import 성공!

🎯 모든 설정이 완료되었습니다!


## 1. OCR 인스턴스 생성

MyPaddleOCR 클래스의 인스턴스를 생성합니다.

In [2]:
ocr = MyPaddleOCR()

/home/aidan/miniconda3/envs/meow-chat/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', No

In [3]:
ocr.get_available_langs()

Available Languages (33 total):
['ch', 'en', 'korean', 'japan', 'chinese_cht', 'ta', 'te', 'ka', 'latin', 'arabic', 'cyrillic', 'devanagari', 'french', 'german', 'it', 'xi', 'pu', 'ru', 'ar', 'hi', 'ug', 'fa', 'ur', 'rs_latin', 'oc', 'rs_cyrillic', 'bg', 'uk', 'be', 'kn', 'ch_tra', 'mr', 'ne']

Major supported languages:
  ch           - 중국어 (간체)
  en           - 영어
  korean       - 한국어
  japan        - 일본어
  chinese_cht  - 중국어 (번체)
  french       - 프랑스어
  german       - 독일어
  ru           - 러시아어
  ar           - 아랍어
  hi           - 힌디어


['ch',
 'en',
 'korean',
 'japan',
 'chinese_cht',
 'ta',
 'te',
 'ka',
 'latin',
 'arabic',
 'cyrillic',
 'devanagari',
 'french',
 'german',
 'it',
 'xi',
 'pu',
 'ru',
 'ar',
 'hi',
 'ug',
 'fa',
 'ur',
 'rs_latin',
 'oc',
 'rs_cyrillic',
 'bg',
 'uk',
 'be',
 'kn',
 'ch_tra',
 'mr',
 'ne']

In [4]:
ocr.get_available_models()

🤖 PaddleOCR 사용 가능한 모델 정보
✅ 실제 다운로드된 모델 (7개):
  🔍 텍스트 감지 모델:
    📦 PP-OCRv5_server_det (PP-OCRv5)
  📝 텍스트 인식 모델:
    📦 en_PP-OCRv5_mobile_rec (PP-OCRv5, 영어, 모바일)
    📦 PP-OCRv5_server_rec (PP-OCRv5, 기타, 서버)
    📦 korean_PP-OCRv5_mobile_rec (PP-OCRv5, 한국어, 모바일)
  📐 방향/구조 보정 모델:
    📦 PP-LCNet_x1_0_doc_ori
    📦 PP-LCNet_x1_0_textline_ori
    📦 UVDoc

📚 PaddleOCR 지원 모델 버전:

#1 PP-OCRv5 ✅ 현재 사용 중
   🌍 지원 언어 (12개): ['ch', 'en', 'korean', 'japan', 'chinese_cht']...
   ⭐ 특징: 최고 정확도, 한국어 최적화, 모바일/서버 지원

#2 PP-OCRv4 🔶 이전 버전
   🌍 지원 언어 (5개): ['ch', 'en', 'korean', 'japan', 'chinese_cht']
   ⭐ 특징: 안정적 성능, 빠른 처리

#3 PP-OCRv3 🔶 이전 버전
   🌍 지원 언어 (12개): ['ch', 'en', 'korean', 'japan', 'chinese_cht']...
   ⭐ 특징: 광범위한 언어 지원

#4 PP-OCRv2 🔸 레거시
   🌍 지원 언어 (1개): ['ch']
   ⭐ 특징: 중국어 전용

💡 현재 설정:
   📌 사용 중인 버전: PP-OCRv5
   📌 주 사용 언어: korean
   📌 모델 구성: 서버급 감지 + 모바일 최적화 인식


['en_PP-OCRv5_mobile_rec',
 'PP-OCRv5_server_rec',
 'PP-LCNet_x1_0_doc_ori',
 'PP-LCNet_x1_0_textline_ori',
 'UVDoc',
 'PP-OCRv5_server_det',
 'korean_PP-OCRv5_mobile_rec']

In [5]:
# 테스트할 이미지 경로
img_path = 'assets/images/test_image_1.jpg'
print(f"분석할 이미지: {img_path}")

분석할 이미지: assets/images/test_image_1.jpg


In [6]:
# OCR 실행 및 결과 출력
result = ocr.run_ocr(img_path, debug=True)

📁 파일에서 OCR 실행: assets/images/test_image_1.jpg
✅ 파일 OCR 완료:
   📝 인식된 텍스트 (6개): ['메이크업존', 'MAKEUP ZONE', '드레스 피팅룸', '포토존', 'DRESS FITTING ROOM', 'PHOTO ZONE']
   📊 신뢰도: ['0.9999', '0.9706', '0.9841', '0.9998', '0.9706', '0.9554']
   📍 좌표 정보: 6개 영역


In [ ]:
# 두 결과의 차이점 비교
print("=== result (run_ocr 반환값) ===")
print(f"타입: {type(result)}")
print(f"내용: {result}")

print("\n=== detailed_result (get_ocr_result 반환값) ===")
detailed_result = ocr.get_ocr_result()
print(f"타입: {type(detailed_result)}")

if isinstance(detailed_result, dict) and 'rec_texts' in detailed_result:
    print("📝 인식된 텍스트:")
    for i, text in enumerate(detailed_result['rec_texts'], 1):
        score = detailed_result['rec_scores'][i-1] if 'rec_scores' in detailed_result else 0
        print(f"  {i}. {text} (신뢰도: {score:.3f})")
    
    print(f"\n📊 상세 정보:")
    print(f"  - 텍스트 개수: {len(detailed_result['rec_texts'])}")
    print(f"  - 좌표 정보: {'rec_polys' in detailed_result}")
    print(f"  - 신뢰도 정보: {'rec_scores' in detailed_result}")
else:
    print("상세 결과가 없습니다.")

print(f"\n🔍 result == detailed_result['rec_texts']: {result == detailed_result.get('rec_texts', [])}")

## 7. 실제 사용 중인 모델 정보 확인 (개선된 버전)

이제 하드코딩이 아닌 실제 PaddleOCR 인스턴스에서 동적으로 모델 정보를 추출합니다.

In [ ]:
# 새로운 OCR 인스턴스로 실제 모델 정보 테스트
print("🔍 새로운 OCR 인스턴스 생성 및 실제 모델 정보 추출")
print("=" * 60)

# 새로운 OCR 인스턴스 생성 (개선된 버전 사용)
# from paddleocr import MyPaddleOCR

print("📋 새로운 MyPaddleOCR 인스턴스 생성 중...")
new_ocr = MyPaddleOCR(lang='korean')

print("✅ 생성 완료!\n")

# 실제 모델 정보 가져오기
current_model_info = new_ocr.get_current_model_info()

print("\n" + "🎯 " + "=" * 55)
print("주요 발견사항:")
print(f"   📌 실제 감지 모델: {current_model_info['detection_model']}")
print(f"   📌 실제 인식 모델: {current_model_info['recognition_model']}")
print(f"   📌 언어 설정: {current_model_info['language']}")

# 안전하게 키 존재 확인
if 'pipeline_config' in current_model_info:
    print(f"   📌 문서 처리: {'고급' if current_model_info['pipeline_config']['use_doc_preprocessor'] else '기본'}")
    print(f"   📌 텍스트라인 보정: {'사용' if current_model_info['pipeline_config']['use_textline_orientation'] else '미사용'}")

if 'doc_orientation_models' in current_model_info and current_model_info['doc_orientation_models']:
    print(f"   📌 문서 보정 모델: {len(current_model_info['doc_orientation_models'])}개")
    for model in current_model_info['doc_orientation_models']:
        print(f"      └─ {model}")

print("=" * 60)

# 기존 OCR과 새 OCR 비교
print("\n🔄 기존 OCR vs 새 OCR 비교:")
print(f"   기존 OCR 타입: {type(ocr)}")
print(f"   새 OCR 타입: {type(new_ocr)}")
print(f"   기존 OCR 언어: {ocr.lang}")
print(f"   새 OCR 언어: {new_ocr.lang}")

# 전역 변수 업데이트
ocr_v2 = new_ocr
print("✅ 새 OCR 인스턴스를 ocr_v2 변수에 저장했습니다.")

In [ ]:
# 모듈 리로드하여 최신 버전 사용
import importlib
import main

# 모듈 리로드
importlib.reload(main)

print("🔄 main 모듈 리로드 완료!")

# 최신 버전으로 새 OCR 인스턴스 생성
print("📋 최신 버전으로 OCR 인스턴스 생성...")
latest_ocr = main.MyPaddleOCR(lang='korean')

print("\n🔍 최신 버전의 get_current_model_info() 실행:")
model_info = latest_ocr.get_current_model_info()

print("\n🎯 완전한 모델 정보 구조:")
import json
print(json.dumps(model_info, indent=2, ensure_ascii=False))